In [1]:
import sys
sys.path.insert(1, '/Users/mac/Desktop/PycharmProjects/TAADL/src')
sys.path.insert(2, '/Users/mac/Desktop/PycharmProjects/TAADL/models')

import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from scipy import stats
from config import DATA_PATH
from typing import Optional, Tuple
from scipy.stats import norm
from torch.distributions.multivariate_normal import MultivariateNormal

In [6]:
#TODO run Optuna
#TODO add mean and covariance to existing RP strategy and do the plotting

In [8]:
Z_tr_hat[0]

tensor([[0.0059, 0.0041, 0.0027, 0.0020, 0.0121, 0.0087, 0.0120]])

In [9]:
Z_tr[te_idx]

tensor([[-0.0312, -0.0376, -0.0032, -0.0074, -0.0311, -0.0300, -0.0178]])